In [ ]:
import pandas as pd
import numpy as np
import re # regex for cleaning the tweets
from nltk.tokenize import TweetTokenizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
!pip install pymorphy2
import spacy
from spacy.lang.ru import Russian
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Сентимент-анализ

In [ ]:
#чтение размеченных датасетов
dfpos = pd.read_excel('positive.xlsx')
dfpos['label'] = 1
dfneg = pd.read_excel('negative.xlsx')
dfneg['label'] = 0
dftagged = pd.concat([dfpos, dfneg], ignore_index=True)

In [ ]:
#функция очистки данных
nlp = Russian()


def my_tokenizer(text):
  text = re.sub("[A-Za-z0-9!#$%&';:*+,./<=>?@[\]^_`()«»...{|}~—\"\-]+","",text)
  stopWords = stopwords.words('russian')
  stopWords.extend(['мтс', 'билайн', 'мегафон', 'компании', 'пожалуйста','бизнес', 'теле', 'это', 'мочь', 'который', 'здравствуйте', 'также', 'вообще', 'компания', 'вымпелком', 'ростелеком', 'тело', 'стать', 'очень', 'г'])
  doc = nlp(text.strip())
  for token in doc:
    print(token, token.lemma, token.lemma_)
  tokens = [token.lemma_ for token in doc]
  words_clear = []
  for w in tokens:
      if w.isalpha() ==True and w.lower() not in stopWords:
          words_clear.append(w.lower())

  return words_clear

In [ ]:
dftagged.rename(columns={"Column4": "text"},inplace=True)
X_train, X_test, y_train, y_test = train_test_split(dftagged['text'], dftagged['label'], test_size=0.2, random_state=42)

In [ ]:
#тест Байесовского классификатора с BOW 
vec = CountVectorizer(ngram_range=(1,2), tokenizer = my_tokenizer)
bow = vec.fit_transform(X_train)
bow_test = vec.transform(X_test)

clf = MultinomialNB()
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

In [ ]:
#тест Байесовского классификатора с TF-IDF
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer = my_tokenizer)
tfidf = vec.fit_transform(X_train)
tfidf_test = vec.transform(X_test)

clf = MultinomialNB()
clf.fit(tfidf, y_train)
pred = clf.predict(tfidf_test)
print(classification_report(y_test, pred))

In [ ]:
#тест SVM с BOW
vec = CountVectorizer(ngram_range=(1,2), tokenizer = my_tokenizer)
bow = vec.fit_transform(X_train)
bow_test = vec.transform(X_test)

clf = svm.SVC()
clf.fit(bow, y_train)
pred = clf.predict(bow_test)
print(classification_report(y_test, pred))

In [ ]:
#тест SVM с TF-IDF
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer = my_tokenizer)
tfidf = vec.fit_transform(X_train)
tfidf_test = vec.transform(X_test)

clf2 = svm.SVC()
clf2.fit(tfidf, y_train)
pred = clf.predict(tfidf_test)
print(classification_report(y_test, pred))

In [ ]:
#чтение неразмеченных датасетов по компаниям
beeline = pd.read_excel('beeline.xlsx', header=None)
megafon = pd.read_excel('megafon.xlsx',header=None)
mts = pd.read_excel('mts.xlsx',header=None)
tele2 = pd.read_excel('tele2.xlsx',header=None)
beeline.rename(columns={beeline.columns[0]: "text"}, inplace=True)
megafon.rename(columns={megafon.columns[0]: "text"}, inplace=True)
mts.rename(columns={mts.columns[0]: "text"}, inplace=True)
tele2.rename(columns={tele2.columns[0]: "text"}, inplace=True)
beeline.rename(columns={beeline.columns[1]: "date"}, inplace=True)
megafon.rename(columns={megafon.columns[1]: "date"}, inplace=True)
mts.rename(columns={mts.columns[1]: "date"}, inplace=True)
tele2.rename(columns={tele2.columns[1]: "date"}, inplace=True)

In [ ]:
#предсказывание классов по компаниям
beeline_bow = vec.transform(beeline.text)
beeline['labels'] = clf.predict(beeline_bow)
megafon_bow = vec.transform(megafon.text)
megafon['labels'] = clf.predict(megafon_bow)
mts_bow = vec.transform(mts.text)
mts['labels'] = clf.predict(mts_bow)
tele2_bow = vec.transform(tele2.text)
tele2['labels'] = clf.predict(tele2_bow)

In [ ]:
#объединение для построения общего графика по классам и компаниям, в работе реализовано в Power BI 
dflab = pd.concat([beeline, megafon, mts, tele2])
beeline['company'] = 'Билайн'
megafon['company'] = 'Мегафон'
mts['company'] = 'МТС'
tele2['company'] = 'Теле2'
fig = plt.figure(figsize = (10, 5))
sns.set_style('white')
sns.set_style('ticks')
plt.title('Overall sentiments')
sns.countplot(x = 'company', hue='labels',  data =dflab, palette = ['red', 'green'])
plt.show()

Тематическое моделирование

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from pprint import pprint
import matplotlib.colors as mcolors

In [ ]:
#предобработка и очистка данных
beeline['posttext'] = beeline.text.apply(my_tokenizer)
mts['posttext'] = mts.text.apply(my_tokenizer)
megafon['posttext'] = megafon.text.apply(my_tokenizer)
tele2['posttext'] = tele2.text.apply(my_tokenizer)

In [ ]:
#создание облака слов по классам для каждой компании на примере негативных отзывов Билайн (Рис 8)
stopWords = set(stopwords.words('russian'))
text = " ".join(str(review) for review in beeline[beeline.labels == 0].posttext)
cloudtext = re.sub("[A-Za-z0-9!#$%&':*+,./<=>?@[\]^_`()«»...{|}~—\"\-]+"," ",text)
wordcloud = WordCloud(stopwords=stopWords, max_words=50, width=1600, height=800).generate(cloudtext)

plt.figure(figsize=(40,30))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
#подготовка к LDA
words = beeline[beeline.labels == 0].posttext.values.tolist()
id2word = corpora.Dictionary(words)
#создание корпуса
texts = words
#матрица частот слов в документах
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
#скачивание пакетов для LDAMallet
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

In [ ]:
mallet_path = '/content/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=6, id2word=id2word)
#отображение сгенерированных тем
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
#составление облаков слов по сгенерированным темам (Рис 12)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  

cloud = WordCloud(
                  background_color='black',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = ldamallet.show_topics(formatted=False)

fig, axes = plt.subplots(3, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
#расчет весов сгенерированных тем в коллекции документов
tm_results = ldamallet[corpus]
corpus_topics = [sorted(topics, key=lambda record: -record[1])[0] for topics in tm_results]
corpus_topic_df = pd.DataFrame()
corpus_topic_df['Dominant Topic'] = [item[0] for item in corpus_topics]
corpus_topic_df['Contribution %'] = [round(item[1]*100, 2) for item in corpus_topics]
dominant_topic_df = corpus_topic_df.groupby('Dominant Topic').agg(
                                  Doc_Count = ('Dominant Topic', np.size),
                                  Topic_Weight = ('Dominant Topic', np.size)).reset_index()

dominant_topic_df['Topic_Weight'] = dominant_topic_df['Topic_Weight'].apply(lambda row: round((row*100) / len(corpus), 2))

In [ ]:
#график распределения весов тем в коллекции для каждой компании и класса на примере негативных отзывов Билайн 
fig = plt.figure(figsize = (10, 5))
sns.set_style('dark')
sns.set_style('ticks')
plt.title('Topics distribution over Beeline Negative Tweets')
sns.barplot(x = 'Dominant Topic', y = 'Topic_Weight',  data =dominant_topic_df, palette = 'magma')

plt.show()